<a href="https://colab.research.google.com/github/garenoo/2023MLProgramming/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 정형데이터 vs 비정형데이터
# 정형데이터: CV 파일처럼 구조를 가지는 것
# 비정형데이터: 덱스트 데이터, 사진, 디지털 음악 등

In [ ]:
# 앙상블 학습
  # 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘
  # 결정트리 기반으로 동작

In [ ]:
# 랜덤 포레스트
  # 앙상블 학습 중 가장 유명하고 안정적인 성능
  # 여러 개의 결정트리를 랜덤하게 만들고 각 결정 트리의 예측을 사용해 최종 예측 출력

  # 훈련하기 위한 데이터를 랜덤하게 만듬
    # 중복으로도 뽑을 수 있는데, 이렇게 만들어진 샘플을 부트스트랩 샘플이라고 함
    # 뽑아지는 샘플의 수는 원래 훈련세트와 동일
  # 부트스트랩 샘플로 결정트리를 훈련
    # 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
    # RandomForestClassifier
    # RandomForestRegressor
  # 정해진 수만큼 위 방법을 반복

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [ ]:
# OOB(Out of Bag) 샘플
  # 부트스트랩 샘플에 포함되지 않는 샘플

In [5]:
# 엑스트라트리
  # RF와 비슷하게 동작
    # 부트스트랩 샘플을 사용하지 않고, 각 결정 트리를 만들 때 전체 훈련 세트 사용
    # 노드를 분할할 때 가장 좋은 분할을 찾지 않고 랜덤으로 결정

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
# 배깅과 부스팅
  # 배깅: 여러 개의 모델을 랜덤하게 만들고 각 모델의 결과를 종합하여 결과를 출력
  # 부스팅: 모델을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모델을 만들고 최종적으로 생성된 모든 모델을 하나로 합침

In [8]:
# 그레이디언트 부스팅
  # 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완
  # 과대적합에 강함, 일반화 성능이 높음
  # 경사하강법 사용하여 트리를 앙상블에 추가
  # 경사하강법이 손실함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정이라면,
  # 그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동함
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [ ]:
# 히스토그램 기반 그레이디언트 부스팅
  # 정형 데이터 다루는 머신러닝 알고리즘 중 제일 인기가 많음
  # 미리 하나 떼어놓고, 누락된 값이 있으면 떼어놓은 걸로 대신 사용
  # 실수 범위를 정수 범위로 축소할 수 있음

  # 기존에는 특성 중요도를 계산하기 위해 feature_importance()를 썼다면, permutation_importances()를 써야함
  # 각 특성별로 값들을 랜덤하게 섞어 3가지 데이터에 대해 모델을 평가

In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

In [15]:
# 사이킷런이 아닌 다른 곳에서 그레이디언트 부스팅 알고리즘을 구현한 라이브러리
  # XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [16]:
  # LightGBM
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
